In [46]:
import random
import time
import functools
import sys
from time import sleep

### Decorators

In [3]:
def send_email(settings=None):
    time.sleep(random.randint(1, 3))

send_email(settings={'smtp': 'smtp.google.com'})
send_email(settings={'smtp': 'smtp.outlook.live.com'})

In [5]:
def dump_database(storage=None):
    time.sleep(random.randint(0, 1))

dump_database(storage='s3')
dump_database(storage='gdrive')

In [13]:
def send_email(settings=None):
    time.sleep(random.randint(1, 3))

start = time.time()
send_email(settings={'smtp': 'smtp.google.com'})
end = time.time()
print(f'Elapsed: {end - start}s')

start = time.time()
send_email(settings={'smtp': 'smtp.outlook.live.com'})
end = time.time()
print(f'Elapsed: {end - start}s')

Elapsed: 2.002208948135376s
Elapsed: 2.0024538040161133s


In [68]:
def send_email(settings=None):
    start = time.time()

    time.sleep(random.randrange(1, 3))

    end = time.time()
    print(f'Elapsed  with {settings}: {end - start}s')

send_email(settings={'smtp': 'smtp.google.com'})
send_email(settings={'smtp': 'smtp.outlook.live.com'})

Elapsed  with {'smtp': 'smtp.google.com'}: 1.0015008449554443s
Elapsed  with {'smtp': 'smtp.outlook.live.com'}: 2.0020358562469482s


In [14]:
def dump_database(storage=None):
    start = time.time()

    time.sleep(random.randrange(0, 1))

    end = time.time()
    print(f'Elapsed with {storage}: {end - start}s')

dump_database(storage='s3')
dump_database(storage='gdrive')

Elapsed with s3: 6.031990051269531e-05s
Elapsed with gdrive: 8.821487426757812e-06s


In [20]:
def some_other_func(*args):
    start = time.time()

    lst = [i for i in range(10**6)]

    print(f'Elapsed {args}: {time.time() - start}s')

some_other_func()

Elapsed (): 0.028357982635498047s


In [ ]:
def profile():
    start = time.time()

    # some code

    print(f'Elapsed: {time.time() - start}s')

In [6]:
def dump_database(storage=None):
    time.sleep(random.randrange(0, 1))
    
def send_email(settings=None):
    time.sleep(random.randint(1, 3))

In [9]:
def profile(f):
    start = time.time()

    f()

    print(f'Elapsed ({f.__name__}): {time.time() - start}s')

profile(dump_database)
profile(send_email)

Elapsed (dump_database): 6.914138793945312e-05s
Elapsed (send_email): 3.002246141433716s


In [11]:
id(id)

139840879265808

In [73]:
def say_hello():
    def internal():
        print('Hello')
    return internal

f = say_hello()
print(type(f))
f()

<class 'function'>
Hello


In [74]:
def say_hello():
    def internal(msg):
        print('Hello', msg)
    return internal

f1 = say_hello()
f1('John')

f1 = say_hello()
f1('Bill')

Hello John
Hello Bill


In [75]:
def say_hello(greeting='Hello'):
    def internal(msg):
        print(greeting, msg)
    return internal

f1 = say_hello()
f1('John')

f1 = say_hello('Hi')
f1('Bill')

Hello John
Hi Bill


In [76]:
def say_hello(greeting='Hello'):
    def internal(*args):
        print(greeting, *args)
    return internal

f1 = say_hello()
f1('John', 42, [1, 2, 3], True)

Hello John 42 [1, 2, 3] True


In [77]:
def profile(f):
    def deco(*args):
        start = time.time()
        f(*args)
        print(f'Elapsed time for function {f.__name__}: {time.time() - start}ms')
    return deco

In [ ]:
def profile(f):
    def deco(*args, **kwargs):
        start = time.time()
        f(*args, **kwargs)
        print(f'Elapsed time for function {f.__name__}: {time.time() - start}ms')
    return deco

In [13]:
def profile(f):
    def deco(*args, **kwargs):
        start = time.time()
        result = f(*args, **kwargs)
        print(f'Elapsed time for function {f.__name__}: {time.time() - start}ms')
        return result
    return deco

In [14]:
# foo()
foo_decorated=profile(send_email)
foo_decorated({'smtp': 'google.com'})
foo_decorated({'smtp': 'live.com'})

Elapsed time for function send_email: 3.002382278442383ms
Elapsed time for function send_email: 1.0010349750518799ms


In [82]:
send_email=profile(send_email)
send_email({'smtp': 'google.com'})
send_email({'smtp': 'live.com'})

Elapsed time for function send_email: 3.001675844192505ms
Elapsed time for function send_email: 2.00132155418396ms


In [18]:
@profile # -> dump_database = profile(dump_database)
def dump_database(storage=None):
    time.sleep(random.randrange(0, 1))
    
@profile
def send_email(settings=None):
    time.sleep(random.randint(1, 3))


dump_database()
send_email()

Elapsed time for function dump_database: 8.082389831542969e-05ms
Elapsed time for function send_email: 1.0008487701416016ms


### We need to go deeper

In [23]:
@profile
def foo7():
    """Help for foo7"""
    time.sleep(random.randint(1, 2))
    return 42

help(foo7)
print(foo7())

Help on function foo7 in module __main__:

foo7()
    Help for foo7

Elapsed time for function foo7: 1.0008957386016846ms
None


In [21]:
def profile(f):
    @functools.wraps(f)
    def deco(*args):
        start = time.time()
        f(*args)
        print(f'Elapsed time for function {f.__name__}: {time.time() - start}ms')
    return deco

In [24]:
@profile
def foo8():
    """Help for foo8"""
    time.sleep(random.randint(1, 2))
    return 42

help(foo8)
print(foo8())

Help on function foo8 in module __main__:

foo8()
    Help for foo8

Elapsed time for function foo8: 1.0005488395690918ms
None


In [30]:
def profile(msg='Elapsed time'):
    def internal(f):
        @functools.wraps(f)
        def deco(*args, **kwargs):
            start = time.time()
            result = f(*args, **kwargs)
            print(msg, f'({f.__name__}): {time.time() - start}s')
            return result
        return deco
    return internal

In [31]:
@profile(msg='Прошло времени')
def dump_database(storage=None):
    time.sleep(random.randrange(0, 1))
    
@profile(msg='Elapsed time')
def send_email(settings=None):
    time.sleep(random.randint(1, 3))

send_email(settings={'smtp': 'smtp.google.com'})
dump_database(storage='s3')

Elapsed time (send_email): 3.0028645992279053s
Прошло времени (dump_database): 0.0004756450653076172s


In [94]:
@profile('Elapsed time') # -> profile_ = profile('Time spent')
                         # -> foo5 = profile_(foo5)
def foo8():
    """Help for foo7"""
    time.sleep(random.randint(1, 2))
    return 42

help(foo8)
print("RESULT: ", foo8())

Help on function foo8 in module __main__:

foo8()
    Help for foo7

Elapsed time foo8: 1.0004870891571045ms
RESULT:  42


### Template

#### 2-level decorator

In [117]:
def profile(f):
    @functools.wraps(f)
    def deco(*args, **kwargs):
        start = time.time()
        result = f(*args, **kwargs)
        print('Elapsed time', f'({f.__name__}): {time.time() - start}s')
        return result
    return deco

@profile
def send_email(): ...

#### 3-level decorator

In [47]:
def profile(msg='Elapsed time', file=sys.stdout):
    def internal(f):
        @functools.wraps(f)
        def deco(*args, **kwargs):
            start = time.time()
            result = f(*args, **kwargs)
            print(msg, f'({f.__name__}): {time.time() - start}s', file=file)
            return result
        return deco
    return internal

@profile()
def send_email(): ...

### Real life examples

In [118]:
# 0 1 2 3 4 5 6 7 8
# 0 1 1 2 3 5 8 13 ...
def fibo(n):
    """Super inefficient fibo function"""
    if n < 2:
        return n
    else:
        return fibo(n-1) + fibo(n-2)
   
fibo(10)

55

In [120]:
@profile
def fibo(n):
    """Super inefficient fibo function"""
    if n < 2:
        return n
    else:
        return fibo(n-1) + fibo(n-2)
   
fibo(10)

Elapsed time (fibo): 1.1920928955078125e-06s
Elapsed time (fibo): 9.5367431640625e-07s
Elapsed time (fibo): 0.0002105236053466797s
Elapsed time (fibo): 7.152557373046875e-07s
Elapsed time (fibo): 0.0002827644348144531s
Elapsed time (fibo): 7.152557373046875e-07s
Elapsed time (fibo): 9.5367431640625e-07s
Elapsed time (fibo): 7.033348083496094e-05s
Elapsed time (fibo): 0.0004215240478515625s
Elapsed time (fibo): 4.76837158203125e-07s
Elapsed time (fibo): 4.76837158203125e-07s
Elapsed time (fibo): 5.984306335449219e-05s
Elapsed time (fibo): 4.76837158203125e-07s
Elapsed time (fibo): 0.00010943412780761719s
Elapsed time (fibo): 0.0005881786346435547s
Elapsed time (fibo): 2.384185791015625e-07s
Elapsed time (fibo): 2.384185791015625e-07s
Elapsed time (fibo): 4.124641418457031e-05s
Elapsed time (fibo): 2.384185791015625e-07s
Elapsed time (fibo): 8.320808410644531e-05s
Elapsed time (fibo): 2.384185791015625e-07s
Elapsed time (fibo): 2.384185791015625e-07s
Elapsed time (fibo): 4.24385070800781

55

#### Recursion support

In [121]:
def profile(msg="Elapsed time for function"):
    def internal(f):
        @functools.wraps(f)
        def deco(*args, **kwargs):
            start = time.time()
            deco._num_call += 1
            result = f(*args, **kwargs)
            deco._num_call -= 1
            
            if deco._num_call == 0:
                print(msg, f'{f.__name__}: {time.time() - start}s')
            return result
        
        deco._num_call = 0
        return deco
    
    return internal

@profile()
def fibo(n):
    """Super inefficient fibo function"""
    if n < 2:
        return n
    else:
        return fibo(n-1) + fibo(n-2)
   
fibo(10)

Elapsed time for function fibo: 0.0001461505889892578s


55

#### Exponential backoff

In [74]:
def repeate(max_repeat=10):
    def internal(f):
        @functools.wraps(f)
        def repeater(*args, **kwargs):
            while repeater._num_repeats <= max_repeat:
                try:
                    return f(*args, **kwargs)
                except Exception as ex:
                    if repeater._num_repeats == max_repeat:
                        raise
                    else:
                        print(f'Failed after {repeater._num_repeats+1} times, trying again after {2**repeater._num_repeats} sec...')
                        sleep(2**repeater._num_repeats)
                        repeater._num_repeats += 1
                
        repeater._num_repeats = 0
        return repeater
    return internal


@repeate(max_repeat=4)
# @repeate()
# @repeate # note the difference
def connect_to_server(*args):
    print('Trying to connect: ', *args)
    if sum(random.choices([0, 1], [0.8, 0.2])) == 0:
        raise RuntimeError('Failed to connect')
    print('SUCCESS!')

connect_to_server('google.com')


Trying to connect:  google.com
Failed after 1 times, trying again after 1 sec...
Trying to connect:  google.com
Failed after 2 times, trying again after 2 sec...
Trying to connect:  google.com
SUCCESS!


#### Cache

In [124]:
def cache(f):
    
    @functools.wraps(f)
    def deco(*args):
        
        if args in deco._cache:
            return deco._cache[args]
        
        result = f(*args)
        
        deco._cache[args] = result
        
        return result
    
    deco._cache = {}
        
    return deco

In [80]:
@profile()
# @cache(max_size=64)
@cache
def foo(n):
    time.sleep(n)

foo(5)
foo(5)
foo(6)
foo(6)

Elapsed time for function foo: 5.003516912460327s
Elapsed time for function foo: 4.0531158447265625e-06s
Elapsed time for function foo: 6.002982139587402s
Elapsed time for function foo: 5.0067901611328125e-06s


In [125]:
@profile()
@cache
def fibo(n):
    """Super inefficient fibo function"""
    if n < 2:
        return n
    else:
        return fibo(n-1) + fibo(n-2)


In [126]:
# for i in range(1000):
#     print(i, '->', fibo(i))
fibo(10)
# fibo(1000)

Elapsed time for function fibo: 3.4809112548828125e-05s


55

In [127]:
fibo._cache

{(1,): 1,
 (0,): 0,
 (2,): 1,
 (3,): 2,
 (4,): 3,
 (5,): 5,
 (6,): 8,
 (7,): 13,
 (8,): 21,
 (9,): 34,
 (10,): 55}

In [128]:
from functools import lru_cache

@lru_cache(maxsize=64)
def foo():
    print('foo')

### Links

1. Python Intorduciton: https://www.youtube.com/watch?v=5V7XG1mGiHc&list=PLlb7e2G7aSpTTNp7HBYzCBByaE1h54ruW
2. Теория по декораторам: http://bit.ly/2z5yatp: стр. 414
3. Примеры с урока: https://github.com/dbradul/python_course/blob/master/lesson_deco.ipynb
4. Exponential backoff: https://bit.ly/3nvOyeQ
5. Установка jupyter: https://www.youtube.com/watch?v=VIaur9G-0tc

### H/w

1. Дополнить декоратор сache поддержкой max_limit.
2. 